In [2]:
import pandas as pd
import numpy as np
import imblearn
import random

In [5]:
n = 10000
skip = sorted(random.sample(range(1, 595212), 595212 - n))
data = pd.read_csv('train.csv', skiprows=skip)
data.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
1,46,0,5,1,11,0,0,0,0,0,...,4,1,3,9,0,0,0,0,1,0
2,121,0,2,1,6,0,0,1,0,0,...,7,1,2,3,0,0,1,0,0,0
3,583,0,0,1,6,1,0,0,1,0,...,5,3,2,6,0,1,1,0,0,0
4,1112,0,7,1,6,0,6,0,0,0,...,3,1,5,5,0,1,1,0,0,0


In [6]:
len(data)

10000

In [7]:
X = data.iloc[:, 2:]
X.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,1,2,0,0,1,0,0,0,0,...,2,2,4,9,0,0,0,0,0,0
1,5,1,11,0,0,0,0,0,1,0,...,4,1,3,9,0,0,0,0,1,0
2,2,1,6,0,0,1,0,0,0,0,...,7,1,2,3,0,0,1,0,0,0
3,0,1,6,1,0,0,1,0,0,0,...,5,3,2,6,0,1,1,0,0,0
4,7,1,6,0,6,0,0,0,1,0,...,3,1,5,5,0,1,1,0,0,0


In [8]:
y = data['target']

In [9]:
cat_col = [i for i in range(0, len(X.columns)) if 'cat' in X.columns[i]]
cat_col

[1, 3, 4, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]

In [12]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks()
X_tl, y_tl = tl.fit_sample(X, y)

In [13]:
np.shape(X_tl)

(9875, 57)

In [14]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
X_sm, y_sm = sm.fit_sample(X, y)

In [15]:
np.shape(X_sm)

(19286, 57)

In [17]:
np.unique(y_sm, return_counts=True)

(array([0, 1]), array([9643, 9643]))

In [19]:
from imblearn.combine import SMOTETomek
smt = SMOTETomek()
X_smt, y_smt = smt.fit_sample(X, y)

In [20]:
np.shape(X_smt)

(19286, 57)

In [23]:
np.unique(y_smt, return_counts=True)

(array([0, 1]), array([9643, 9643]))

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [24]:
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, eval_metric='Accuracy', verbose=10, task_type='GPU')

In [25]:
model.fit(X_train, y_train, cat_col, eval_set=(X_test, y_test))

0:	learn: 0.9633318	test: 0.9642144	best: 0.9642144 (0)	total: 190ms	remaining: 18.8s
10:	learn: 0.9633318	test: 0.9642144	best: 0.9642144 (0)	total: 2.01s	remaining: 16.2s
20:	learn: 0.9633341	test: 0.9642144	best: 0.9642144 (0)	total: 3.73s	remaining: 14s
30:	learn: 0.9633341	test: 0.9642144	best: 0.9642212 (28)	total: 5.55s	remaining: 12.4s
40:	learn: 0.9633430	test: 0.9642144	best: 0.9642212 (28)	total: 7.39s	remaining: 10.6s
50:	learn: 0.9633587	test: 0.9642144	best: 0.9642212 (28)	total: 9.08s	remaining: 8.73s
60:	learn: 0.9633610	test: 0.9642144	best: 0.9642212 (28)	total: 10.9s	remaining: 6.98s
70:	learn: 0.9633610	test: 0.9642212	best: 0.9642212 (28)	total: 12.9s	remaining: 5.27s
80:	learn: 0.9633610	test: 0.9642212	best: 0.9642212 (28)	total: 14.6s	remaining: 3.42s
90:	learn: 0.9633610	test: 0.9642279	best: 0.9642279 (87)	total: 16.3s	remaining: 1.61s
99:	learn: 0.9633632	test: 0.9642144	best: 0.9642279 (87)	total: 17.7s	remaining: 0us
bestTest = 0.9642278717
bestIteration = 

In [26]:
y_pred = model.predict(X_test)

In [27]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
cm

array([[143476,      2],
       [  5321,      4]])

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    143478
           1       0.67      0.00      0.00      5325

    accuracy                           0.96    148803
   macro avg       0.82      0.50      0.49    148803
weighted avg       0.95      0.96      0.95    148803



In [32]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_sample(X, y)

In [33]:
np.shape(X_rus)

(43388, 57)

In [36]:
np.unique(y_rus, return_counts=True)

(array([0, 1]), array([21694, 21694]))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus)

In [39]:
model = CatBoostClassifier(iterations=300, learning_rate=0.1, depth=6, eval_metric='Accuracy', verbose=10, task_type='GPU')

In [40]:
model.fit(X_train, y_train, cat_col, eval_set=(X_test, y_test))

0:	learn: 0.5678375	test: 0.5652254	best: 0.5652254 (0)	total: 80.6ms	remaining: 24.1s
10:	learn: 0.5803141	test: 0.5692818	best: 0.5696506 (8)	total: 839ms	remaining: 22s
20:	learn: 0.5909161	test: 0.5777634	best: 0.5777634 (20)	total: 1.58s	remaining: 21s
30:	learn: 0.5964783	test: 0.5802526	best: 0.5811745 (28)	total: 2.3s	remaining: 20s
40:	learn: 0.6009957	test: 0.5813589	best: 0.5814511 (33)	total: 3.08s	remaining: 19.5s
50:	learn: 0.6031468	test: 0.5846778	best: 0.5848622 (49)	total: 3.94s	remaining: 19.2s
60:	learn: 0.6063120	test: 0.5856919	best: 0.5869826 (58)	total: 4.85s	remaining: 19s
70:	learn: 0.6079408	test: 0.5867982	best: 0.5869826 (58)	total: 5.7s	remaining: 18.4s
80:	learn: 0.6102455	test: 0.5871670	best: 0.5882733 (78)	total: 6.47s	remaining: 17.5s
90:	learn: 0.6114440	test: 0.5885498	best: 0.5885498 (90)	total: 7.24s	remaining: 16.6s
100:	learn: 0.6140561	test: 0.5888264	best: 0.5903937 (95)	total: 7.99s	remaining: 15.8s
110:	learn: 0.6150088	test: 0.5892874	best:

In [41]:
y_pred = model.predict(X_test)

In [42]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.60      0.61      0.61      5488
           1       0.59      0.57      0.58      5359

    accuracy                           0.59     10847
   macro avg       0.59      0.59      0.59     10847
weighted avg       0.59      0.59      0.59     10847



In [43]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_sample(X, y)

In [44]:
np.shape(X_ros)

(1147036, 57)

In [45]:
np.unique(y_ros, return_counts=True)

(array([0, 1]), array([573518, 573518]))

In [46]:
del X
del y
del X_rus
del y_rus
del data
del model
del X_train
del X_test
del y_train
del y_test 

In [47]:
model = CatBoostClassifier(iterations=300, learning_rate=0.1, depth=6, eval_metric='Accuracy', verbose=10, task_type='GPU')

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros)

In [49]:
model.fit(X_train, y_train, cat_col, eval_set=(X_test, y_test))

0:	learn: 0.5935135	test: 0.5987153	best: 0.5987153 (0)	total: 2.19s	remaining: 10m 54s
10:	learn: 0.6166944	test: 0.6227773	best: 0.6227773 (10)	total: 25s	remaining: 10m 56s
20:	learn: 0.6371587	test: 0.6440426	best: 0.6440426 (20)	total: 47.2s	remaining: 10m 26s
30:	learn: 0.6542718	test: 0.6619426	best: 0.6619426 (30)	total: 1m 9s	remaining: 10m 1s
40:	learn: 0.6689101	test: 0.6789883	best: 0.6789883 (40)	total: 1m 30s	remaining: 9m 29s
50:	learn: 0.6780072	test: 0.6888677	best: 0.6888677 (50)	total: 1m 53s	remaining: 9m 12s
60:	learn: 0.6843854	test: 0.6948309	best: 0.6948309 (60)	total: 2m 14s	remaining: 8m 45s
70:	learn: 0.6915982	test: 0.7028690	best: 0.7028690 (70)	total: 2m 36s	remaining: 8m 23s
80:	learn: 0.6975718	test: 0.7089193	best: 0.7089193 (80)	total: 2m 57s	remaining: 8m
90:	learn: 0.7022424	test: 0.7145652	best: 0.7145652 (90)	total: 3m 20s	remaining: 7m 39s
100:	learn: 0.7085369	test: 0.7204517	best: 0.7204517 (100)	total: 3m 42s	remaining: 7m 18s
110:	learn: 0.714

In [50]:
y_pred = model.predict(X_test)

In [51]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77    143093
           1       0.77      0.79      0.78    143666

    accuracy                           0.77    286759
   macro avg       0.78      0.77      0.77    286759
weighted avg       0.78      0.77      0.77    286759

